### Journal Papers

    [1] Delia Rusu, Lorand Dali, Blaz Fortuna, Marko Grobelnik, DunjaMladenic, “Triplet extraction from sentences” in Artificial Intelligence Laboratory, Jožef Stefan Institute, Slovenia, Nov. 7, 2008. http://ailab.ijs.si/dunja/SiKDD2007/Papers/Rusu_Trippels.pdf

    [2] The Multi-Liaison algorithm by Ms. Anjali Ganesh Jivani, Ms.AmishaHetalShingala, Dr. Paresh. V. Virparia published in International Journal of Advanced Computer Science and Applications Vol. 2, No. 5, 2011. http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.625.507&rep=rep1&type=pdf

[1] - https://www.researchgate.net/publication/228905420_Triplet_extraction_from_sentences

[2] - https://www.researchgate.net/publication/215498892_The_Multi-Liaison_Algorithm

### Others


stanford corenlp - https://stanfordnlp.github.io/CoreNLP/index.html

dependencies manual - https://nlp.stanford.edu/software/dependencies_manual.pdf

reference - http://shodhganga.inflibnet.ac.in/bitstream/10603/34713/14/14_chapter%206.pdf

In [1]:
import nltk, pandas as pd, numpy as np
from nltk.parse.corenlp import CoreNLPParser, CoreNLPDependencyParser
from nltk.tree import ParentedTree
import re

In [2]:
dep_parser = CoreNLPDependencyParser(url='http://0.0.0.0:9000')
pos_tagger = CoreNLPParser(url='http://0.0.0.0:9000', tagtype='pos')

In [3]:
def convert_sentence (input_sent):
    # Parse sentence using Stanford CoreNLP Parser
    pos_type = pos_tagger.tag(input_sent.split())
    parse_tree, = ParentedTree.convert(list(pos_tagger.parse(input_sent.split()))[0])
    dep_type, = ParentedTree.convert(dep_parser.parse(input_sent.split()))
    return pos_type, parse_tree, dep_type

def multi_liaison (input_sent, output=['tagging','parse_tree','type_dep','spo','relation']):
    pos_type, parse_tree, dep_type = convert_sentence(input_sent)
    pos_sent = ' '.join([x[0]+'/'+x[1] for x in pos_type])
    # Extract subject, predicate and object
    subject, adjective = get_subject(parse_tree)
    predicate = get_predicate(parse_tree)
    objects = get_object(parse_tree)
    # Generate the relations between subjects and objects
    relation = get_relationship(dep_type, subject, predicate, objects)
    if 'tagging' in output:
        print('---TAGGING---')
        print(pos_sent)
        print()
    if 'parse_tree' in output:
        print('---PARSE TREE---')
        parse_tree.pretty_print()
        print()
    if 'type_dep' in output:
        print('---TYPED DEPENDENCIES---')
        for x in dep_type.triples(): print(x) 
        print()
    if 'spo' in output:
        print('---MULTI-LIAISON OUTPUT---')
        print('Subject: ',len(subject))
        for x in subject: print(' '.join(x))
        print('Predicate: ',len(predicate))
        for x in predicate: print(' '.join(x))
        print('Object: ',len(objects))
        for x in objects: print(' '.join(x))
        print()
    if 'relation' in output:
        print('---RELATIONSHIP---')
        for x in relation: print(x)

def get_subject (parse_tree):
    # Extract the nouns and adjectives from NP_subtree which is before the first / main VP_subtree
    subject, adjective = [],[]
    for s in parse_tree:
        if s.label() == 'NP':
            for t in s.subtrees(lambda y: y.label() in ['NN','NNP','NNS','NNPS','PRP']):
                # Avoid empty or repeated values
                if t.pos()[0] not in subject:
                    subject.append(t.pos()[0])
            for t in s.subtrees(lambda y: y.label().startswith('JJ')):
                if t.pos()[0] not in adjective:
                    adjective.append(t.pos()[0])
    return subject, adjective

def get_predicate (parse_tree):
    # Extract the verbs from the VP_subtree
    predicate = []
    for s in parse_tree.subtrees(lambda x: x.label() == 'VP'):
        for t in s.subtrees(lambda y: y.label().startswith('VB')):
            if t.pos()[0] not in predicate:
                predicate.append(t.pos()[0]) 
    return predicate

def get_object (parse_tree):
    # Extract the nouns from VP_NP_subtree
    objects, output = [],[]
    for s in parse_tree.subtrees(lambda x: x.label() == 'VP'):
        for t in s.subtrees(lambda y: y.label() == 'NP'):
            for u in t.subtrees(lambda z: z.label() in ['NN','NNP','NNS','NNPS','PRP$']):
                output = u.pos()[0]
                if u.left_sibling() is not None and u.left_sibling().label().startswith('JJ'):
                    output += u.left_sibling().pos()[0]
                if output not in objects:
                    objects.append(output)
    return objects

def get_relationship (dep_type, subject, predicate, objects):
    # Generate relations based on the relationship dependencies obtained from parse_tree.triples()
    subject = [x[0] for x in subject]
    predicate = [x[0] for x in predicate]
    objects = [x[0] for x in objects]     
    d1, d2, r1, r2, relation, s1, s2, subjs = [],[],[],[],[],[],[],[]
    w1, w2, output = '','',''
    for head, rel, dep in dep_type.triples():
        if rel in ['nsubj','acl:relcl','conj']:
            s1, s2 = head[0], dep[0]
            if s2 in subject and s1 in predicate:
                w1, w2 = s2, s1
            elif s2 in predicate and (s1 in subject or s1 in objects):
                w1, w2 = s1, s2
            elif s2 in subject and s1 in subject:
                subjs = [s1, s2]
            if w1 != '' and w2 != '':
                r1 = [w1, w2]
        if rel in ['dobj','prep','nmod','conj']:
            d1, d2 = head[0], dep[0]
            if d1 in objects and d2 in objects: 
                r2 = [d1,d2]
            elif d2 in objects:
                r2 = [d2]
            elif d1 in objects:
                r2 = [d1]
        if len(r1) != 0 and len(r2) != 0 and (r2[0] not in r1 and r2[-1] not in r1):
            if len(subjs) != 0:
                for n in subjs:
                    output = '-'.join([n] + r1[-1:] + r2)
                    if output not in relation:
                        relation.append(output)
            else:
                output = '-'.join(r1+r2)
                if output not in relation:
                    relation.append(output)  
    rm = [x for x in relation for y in relation if x != y and re.match(x,y)]
    for x in rm: relation.remove(x)    
    return relation

In [4]:
multi_liaison('The old beggar ran after the rich man who was wearing a black coat')

---TAGGING---
The/DT old/JJ beggar/NN ran/VBD after/IN the/DT rich/JJ man/NN who/WP was/VBD wearing/VBG a/DT black/JJ coat/NN

---PARSE TREE---
                S                                                        
      __________|_________                                                
     |                    VP                                             
     |           _________|____________                                   
     |          |                      PP                                
     |          |     _________________|____                              
     |          |    |                      NP                           
     |          |    |         _____________|_____                        
     |          |    |        |                  SBAR                    
     |          |    |        |         __________|_____                  
     |          |    |        |        |                S                
     |          |    |        |      

In [5]:
multi_liaison('The dog and the cat ran after the mouse and the mongoose')

---TAGGING---
The/DT dog/NN and/CC the/DT cat/NN ran/VBD after/IN the/DT mouse/NN and/CC the/DT mongoose/NN

---PARSE TREE---
                             S                                          
              _______________|____                                       
             |                    VP                                    
             |                ____|_____________                         
             |               |                  PP                      
             |               |     _____________|____                    
             |               |    |                  NP                 
             |               |    |         _________|_______            
             NP              |    |        NP        |       |          
      _______|_______        |    |     ___|____     |       |           
     NP      |       NP      |    |    |        NP   |       NP         
  ___|___    |    ___|___    |    |    |        |    |    ___|____

In [6]:
multi_liaison('Jack and I visited the zoo with our children')

---TAGGING---
Jack/NNP and/CC I/PRP visited/VBD the/DT zoo/NN with/IN our/PRP$ children/NNS

---PARSE TREE---
                      S                                
       _______________|_______                          
      |                       VP                       
      |          _____________|________                 
      NP        |         |            PP              
  ____|___      |         |        ____|____            
 NP   |   NP    |         NP      |         NP         
 |    |   |     |      ___|___    |     ____|_____      
NNP   CC PRP   VBD    DT      NN  IN  PRP$       NNS   
 |    |   |     |     |       |   |    |          |     
Jack and  I  visited the     zoo with our      children


---TYPED DEPENDENCIES---
(('visited', 'VBD'), 'nsubj', ('Jack', 'NNP'))
(('Jack', 'NNP'), 'cc', ('and', 'CC'))
(('Jack', 'NNP'), 'conj', ('I', 'PRP'))
(('visited', 'VBD'), 'dobj', ('zoo', 'NN'))
(('zoo', 'NN'), 'det', ('the', 'DT'))
(('visited', 'VBD'), 'nmod', ('chi

### Triplets Extraction from Sentences Example

In [7]:
multi_liaison('A rare black squirrel has become a regular visitor to a suburban garden')

---TAGGING---
A/DT rare/JJ black/JJ squirrel/NN has/VBZ become/VBN a/DT regular/JJ visitor/NN to/TO a/DT suburban/JJ garden/NN

---PARSE TREE---
                  S                                                               
      ____________|_________________                                               
     |                              VP                                            
     |                    __________|_____________                                 
     |                   |                        VP                              
     |                   |     ___________________|_____________                   
     |                   |    |           |                     PP                
     |                   |    |           |              _______|_____             
     NP                  |    |           NP            |             NP          
  ___|____________       |    |      _____|_______      |    _________|_______     
 DT  JJ    JJ     NN